In [2]:
%pip install datasets transformers torch evaluate nltk rouge_score scikit-learn

     ---------------------------------------- 0.0/60.4 kB ? eta -:--:--
     ------------------- ------------------ 30.7/60.4 kB 640.0 kB/s eta 0:00:01
     -------------------------------------- 60.4/60.4 kB 796.3 kB/s eta 0:00:00
   ---------------------------------------- 0.0/9.2 MB ? eta -:--:--
   ---------------------------------------- 0.1/9.2 MB 2.6 MB/s eta 0:00:04
    --------------------------------------- 0.2/9.2 MB 3.4 MB/s eta 0:00:03
   - -------------------------------------- 0.4/9.2 MB 3.0 MB/s eta 0:00:03
   -- ------------------------------------- 0.6/9.2 MB 3.5 MB/s eta 0:00:03
   --- ------------------------------------ 0.8/9.2 MB 3.7 MB/s eta 0:00:03
   ---- ----------------------------------- 1.1/9.2 MB 4.0 MB/s eta 0:00:03
   ----- ---------------------------------- 1.3/9.2 MB 4.0 MB/s eta 0:00:02
   ------ --------------------------------- 1.5/9.2 MB 4.2 MB/s eta 0:00:02
   ------- -------------------------------- 1.8/9.2 MB 4.4 MB/s eta 0:00:02
   -------- ---

In [2]:
from datasets import load_dataset
from transformers import (
    AutoTokenizer,
    AutoModelForTokenClassification,
    TrainingArguments,
    Trainer,
)
import numpy as np
import evaluate

# Setup evaluation
metric = evaluate.load("accuracy")

# checkpoint = "microsoft/codebert-base"
checkpoint = "bert-base-uncased"
dataset_name = "microsoft/LCC_csharp"

# Prepare and tokenize dataset
dataset = load_dataset(dataset_name)
tokenizer = AutoTokenizer.from_pretrained(checkpoint)


def tokenize_function(examples):
    return tokenizer(examples["context"], padding="max_length", truncation=True)


tokenized_datasets = dataset.map(tokenize_function, batched=True)

small_train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(200))
small_eval_dataset = tokenized_datasets["test"].shuffle(seed=42).select(range(200))


def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)


# Load pretrained model and evaluate model after each epoch
model = AutoModelForTokenClassification.from_pretrained(checkpoint, num_labels=5)
training_args = TrainingArguments(
    output_dir="test_trainer", evaluation_strategy="epoch"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=small_train_dataset,
    eval_dataset=small_eval_dataset,
    compute_metrics=compute_metrics,
)

trainer.train()

c:\Users\shafi\source\Notebooks\.venv\Lib\site-packages\datasets\table.py:1421: FutureWarning: promote has been superseded by mode='default'.
  table = cls._concat_blocks(blocks, axis=0)
Map: 100%|██████████| 10000/10000 [00:17<00:00, 578.32 examples/s]
Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


ValueError: The model did not return a loss from the inputs, only the following keys: logits. For reference, the inputs it received are input_ids,token_type_ids,attention_mask.